# Article Important Links
Algorithms to atribute article links importance scores 

KEEP WORKING ON THIS FORMALIZATION AND COMPILATION OF TECHNICS TO COMPUTE IMPORTANT LINKS
MAYBE EXPERIMENT WIKISYNS BASED ON BACKLINKS
EXPERIMENT VARIATIONS OF WIKISYNS 
COMPUTE WORD IMPORTANCE BY ITS SPREAD IN THE ARTICLE (MAYBE TO THIS IN OTHER NOTEBOOK SINCE THIS ONE IS FOR LINKS)
CREATE TREE WITH DICT SYMS

YOU COULD HAVE A LINK STRUCTURE BASED ON UNSUPERVISIED MODELS AND NLP BASED ON SUPERVISED MODELS
THE LINK STRUCTURE WOULD GIVE A BROAD PATH, MEANING THAT DEEP TRANSVERSAL WOULD MEAN SOMETHING YET, BUT NLP BASED WOULD ONLY BE LOCAL
WE COULD USE WIKISYN TO FIND "POSSIBLE LINKS" THAT WERE NOT CREATED

FOR SUPERVISED WE MAY USE:
    NPM DATA
    WORDNET (AND OTHERS) DICTIONARIES
    SCHOOL AND COURSES CURRICULUMS 

LINKS PROBABILITIES ARE:
    BIDIR PROBS
    NUMBER OF LINKS PER PAGE PROB
    WORD COUNT BASED PROB
    
    USE TRELLO TO ORGANIZE EVERTYHING
    PLAN NEXT STEPS
    MELHOR FEITO DO QUE PERFEITO
    
    

In [2]:
from wikipydia import dict_storage, wikidb, parse, wikisyn
WikisynDict = wikisyn.WikisynDict
from collections import Counter
from tabulate import tabulate
import re

In [3]:
def print_data(headers, data_list):
    """Function to print data tabulated """
    return print(tabulate(data_list, headers=headers))
#print_data(['seedrank', 'pagerank', 'in_degree', 'ufauisfha', 'isfudnifsu'], [[1,2,3,4]])

In [4]:
wiki_db = wikidb.WikiDb()

In [5]:
def print_list(iterable):
    for item in iterable:
        print(item)

In [6]:
arti, _ = wiki_db.get_article_by_href("Fluid")
#for link in arti.links():
    #print(link)

### Abstract Links only
Get only the abstract links (first section) and set the same proportional score for all of them

In [7]:
def get_abstract_links(href):
    wikiart, _ = wiki_db.get_article_by_href(href)
    sections = parse._split_html_h2_sections(wikiart.html())
    abstract_links_with_text = parse._filter_wiki_links(parse._extract_html_links(sections[0]))
    
    abstract_links = set([href for href, text in abstract_links_with_text])
    
    all_score = 1 / len(abstract_links)
    
    abstract_links_scores = [(href, all_score) for href in abstract_links]
    
    return abstract_links_scores

In [8]:
#abstract_links = get_abstract_links("Computer")
#print(len(abstract_links))
#print_list(abstract_links[:20])
#print_data(['Article', 'Score'], abstract_links[:20])

### Simple link occurrence count
Just count the number of times some link occurs and score with it. 


There is a problem with this approach that each link may be different but will point to the same thing. In order to detect these links we have to download every one to verify if they point to the same think. 

This issue will appear in other approachs

In [9]:
def simple_link_count(href):
    wikiart, _ = wiki_db.get_article_by_href(href)
    links = wikiart.links()
    
    link_counter = Counter()
    
    for href, text in links:
        link_counter[href] += 1
        
    scores_sum = sum(link_counter.values())
        
    return list(map(lambda a: (a[0], a[1]/scores_sum), link_counter.most_common()))

#print_list(simple_link_count("Computer")[:20])
#print_data(['Article', 'Score'], simple_link_count("Computer")[:20])

### Count local wikisyn
Compute a wikisyn collection with the local links and score links with them.


The wikisyn class computes every href tag text (< a>text< /a>) with their links in order to find link-synonims, that is diferent texts that points to the same links.

Variations to this method may be the use of global wikisyn which computes links syns based on other pages references to the links present on the target article improving the accuracy.

In [10]:
def get_href_text_count(href):
    synDict = WikisynDict()
    
    wikiart, _ = wiki_db.get_article_by_href(href)
    links = wikiart.links()
    page_text = wikiart.text()
    
    #Populate syndict
    for href, text in links:
        synDict.add_link(href, text)
        
    #Get matches
    links_score = Counter()
        
    for link_href in synDict.keys():
        for l_text, l_score in synDict[link_href]:
            matches = re.findall('[^a-zA-Z0-9_]' + re.escape(l_text) + '[^a-zA-Z0-9_]', page_text, re.IGNORECASE)
            matches_score = len(matches) * l_score
            links_score[link_href] += matches_score

    scores_sum = sum(links_score.values())
            
    #norm_links_scores = list(map(lambda a: (a[0],synDict[a[0]], a[1]/scores_sum), links_score.most_common()))
    #norm_links_scores = list(map(lambda a: (a[0], a[1]/scores_sum), links_score.most_common()))
    norm_links_scores = list(map(lambda a: (a[0],list(map(lambda b: b[0], synDict[a[0]])), a[1]/scores_sum), links_score.most_common()))
    
    return norm_links_scores

### Benchmark

In [11]:
#target_article = "Django_(web_framework)"
target_article = "Fluid"

print("SIMPLE WIKISYN COUNT")
print_data(['Article','Link','Score'],get_href_text_count(target_article)[:20]) 
print("\nSIMPLE LINK COUNT")
print_data(['Article', 'Score'], simple_link_count(target_article)[:20])
print("\nAbstract Links")
print_data(['Article', 'Score'], get_abstract_links(target_article))

wiki_db.save()
#USE AUTOSUMARIZER IDEA TO FIND IMPORTANT AREAS OF THE TEXT
#THE MORE SPREAD ARE SIMILAR WORDS, THE MORE IMPORTANT THEY ARE TO UNDERSTAND THE ENTIRE CONTENT
#DIVIDE THE ARTICLE INTO WINDOWS TO CALCULATE THIS

SIMPLE WIKISYN COUNT
Article                         Link                                    Score
------------------------------  ----------------------------------  ---------
Shear_stress                    ['shear stress']                    0.0888889
Stress_(physics)                ['shear stress']                    0.0888889
Strain_(materials_science)      ['strain']                          0.0740741
Physics                         ['physics']                         0.0444444
Liquid                          ['liquids', 'liquid']               0.0444444
Viscosity                       ['viscosity']                       0.0444444
Pressure                        ['pressure']                        0.0296296
Phase_(matter)                  ['phases']                          0.0296296
Plasticity_(physics)            ['plastic solids', 'solids']        0.0296296
Newtonian_fluid                 ['Newtonian fluids']                0.0296296
Compressive_stress              ['compressi

In [11]:
def get_links_score(page):
    """Function to cross a list of links with a text, setting scores."""
    
    wikiart = get_wiki_article_by_href(page)
    
    pageid = wikiart.page_id
    #Ensure only one href is present
    page_links = set([link_href for link_href, link_text in wikiart.links()])
    page_text = wikiart.text()
    
    links_score = dict()
    
    norm_fact = 0 #norm factor to results sum to one
    
    #Ensure all page links are present
    #check_and_download(page_links)
    
    #The ideia is to get for each href the texts that use to follow these hrefs in wikipedia articles.
    #The more each term appears, it got increased weight.
    #Then we try to match each of the terms for each href to the wiki text, applying to each match the correspondent weight.
    #The weight stuff is good to avoid cases where the href_text have appeared only once in one article,
    #but it is a frequent term in other articles but offen does not mean the original href it pointed to
    
    for link_href in page_links:
        links_score[link_href] = 0
        for l_text, l_score in synbeta.get_synoms(link_href): #get_all_wikisyns_by_href(link_href): #wikisyn[link_href]:
            matches = re.findall('[^a-zA-Z0-9_]' + re.escape(l_text) + '[^a-zA-Z0-9_]', page_text, re.IGNORECASE)
            matches_score = len(matches) * l_score
            links_score[link_href] += matches_score
            norm_fact += matches_score
            
    norm_links_score = dict(map(lambda a: [a[0], float(a[1])/norm_fact], links_score.items()))
            
    return norm_links_score

#v_sum = 0
#links_score = get_links_score("Convolutional neural network")
#for k, v in sorted(links_score.items(), key=lambda a:a[1], reverse=True):
    #print(k,synbeta.hrefs[k],v)
    #v_sum += v
#print v_sum

In [12]:
#get_href_text_count("MQTT")